<a href="https://colab.research.google.com/github/gokhangunal/Rating_Product_and_Sorting_Reviews/blob/main/Rating_Products_and_Sorting_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
###################################################
# Rating Products
###################################################

# - Average
# - Time-Based Weighted Average
# - User-Based Weighted Average
# - Weighted Rating

In [2]:
############################################
# Project: Users and Time Based Weighted Average Calculation
############################################

In [3]:
import pandas as pd
import math
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [5]:
df = pd.read_csv("amazon_review.csv")
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0


In [6]:
df.shape

(4915, 12)

In [8]:
# Step 1: Calculate the product's average rating.
df["overall"].mean()

4.587589013224822

In [9]:
# Step 2: Calculate the weighted average rating based on the date.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB


In [13]:
df["reviewTime"] = pd.to_datetime(df["reviewTime"])

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   reviewerID      4915 non-null   object        
 1   asin            4915 non-null   object        
 2   reviewerName    4914 non-null   object        
 3   helpful         4915 non-null   object        
 4   reviewText      4914 non-null   object        
 5   overall         4915 non-null   float64       
 6   summary         4915 non-null   object        
 7   unixReviewTime  4915 non-null   int64         
 8   reviewTime      4915 non-null   datetime64[ns]
 9   day_diff        4915 non-null   int64         
 10  helpful_yes     4915 non-null   int64         
 11  total_vote      4915 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(6)
memory usage: 460.9+ KB


In [17]:
# So how can I access the comments made in this dataframe at certain day intervals?
df.loc[(df["day_diff"] < 30), "overall"].mean()

4.742424242424242

In [18]:
df.loc[(df["day_diff"] > 30) & (df["day_diff"] <= 90), "overall"].mean()

4.803149606299213

In [19]:
df.loc[(df["day_diff"] > 90) & (df["day_diff"] <= 180), "overall"].mean()

4.649484536082475

In [20]:
df.loc[(df["day_diff"] > 180), "overall"].mean()

4.573373327180434

In [22]:
# Since I wanted to increase the weights of the last days, I set a weight of 100%
# and multiplied the average of the scores and comments received on the current dates by 28%.
df.loc[df["day_diff"] <= 30, "overall"].mean() * 29/100 + \
    df.loc[(df["day_diff"] > 30) & (df["day_diff"] <= 90), "overall"].mean() * 27/100 + \
    df.loc[(df["day_diff"] > 90) & (df["day_diff"] <= 180), "overall"].mean() * 24/100 + \
    df.loc[(df["day_diff"] > 180), "overall"].mean() * 20/100

4.702704378099698

In [26]:
# Step 3: Compare and interpret the average of each time period in the weighted rating.
def time_based_weighted_average(dataframe, w1=29, w2=27, w3=24, w4=20):
    return dataframe.loc[df["day_diff"] <= 30, "overall"].mean() * w1 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > 30) & (dataframe["day_diff"] <= 90), "overall"].mean() * w2 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > 90) & (dataframe["day_diff"] <= 180), "overall"].mean() * w3 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > 180), "overall"].mean() * w4 / 100

In [27]:
time_based_weighted_average(df)

4.702704378099698

In [28]:
time_based_weighted_average(df, 30, 26, 22, 22)

4.700574900282908

In [29]:
time_based_weighted_average(df, 28, 26, 24, 22)

4.6987161061560725

In [ ]:
#### Step 1: Generate the helpful_no variable.

# - total_vote is the total number of up-down votes a review receives.
# - up means helpful.
# - There is no helpful_no variable in the dataset; it needs to be generated from existing variables.
# - Subtract the number of helpful votes (helpful_yes) from the total vote count (total_vote)
#to find the number of unhelpful votes (helpful_no).

In [30]:
df.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0


In [31]:
df['helpful_no'] = df['total_vote'] - df['helpful_yes']

In [35]:
# Filter the DataFrame to show only rows where 'helpful_no' is not equal to 0
df[df['helpful_no'] != 0].head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no
17,A1OATGH4DF0RF0,B007WTAJTO,Aaron F. Virginie,"[0, 1]",Class 10 Speed Rating for Seamless Full HD Vid...,5.00000,Get Fast Load Times,1372896000,2013-04-07,610,0,1,1
26,A1U9QJQW3XPRL2,B007WTAJTO,Aaron T. Swain,"[1, 2]","I just received my card, it is the class 10 64...",5.00000,64 GB,1343260800,2012-07-26,865,1,2,1
43,A1X1FX3NSOFCT3,B007WTAJTO,"Abused Commuter ""abused_commuter""","[0, 1]",Ordered this for a Galaxy S3. Lasted a few mo...,1.00000,Beware... goes bad and takes your pics with it,1374278400,2013-07-20,506,0,1,1
70,AR4KV2Q28151J,B007WTAJTO,A discerning buyer,"[0, 1]",How do I praise extra memory? It fills my MP3 ...,5.00000,"Good price, great product",1376956800,2013-08-20,475,0,1,1
75,A2KJY8F5O1U5D6,B007WTAJTO,A. Felson,"[1, 2]",works great on my sprint htc evo 4g lte. Form...,5.00000,works great with htc evo 4g lte,1350259200,2012-10-15,784,1,2,1


In [ ]:
#### Step 2: Calculate and add the scores for score_pos_neg_diff, score_average_rating, and wilson_lower_bound to the data.

# - Define functions for score_pos_neg_diff, score_average_rating, and wilson_lower_bound to calculate these scores.
# - Create scores according to score_pos_neg_diff, then save them in the dataframe as score_pos_neg_diff.
# - Create scores according to score_average_rating, then save them in the dataframe as score_average_rating.
# - Create scores according to wilson_lower_bound, then save them in the dataframe as wilson_lower_bound.


In [ ]:
###################################################
# Yes-No Diff Score = (yes ratings) − (no ratings)
###################################################

In [38]:
def score_pos_neg_diff(helpful_yes, helpful_no):
    return helpful_yes - helpful_no

In [39]:
###################################################
# Score = Average rating = (up ratings) / (all ratings)
###################################################

In [40]:
def score_average_rating(helpful_yes, helpful_no):
    if helpful_yes + helpful_no == 0:
        return 0
    return helpful_yes / (helpful_yes + helpful_no)

In [41]:
def wilson_lower_bound(helpful_yes, helpful_no, confidence=0.95):
    """
    Wilson Lower Bound Score hesapla

    - Bernoulli parametresi p için hesaplanacak güven aralığının alt sınırı WLB skoru olarak kabul edilir.
    - Hesaplanacak skor ürün sıralaması için kullanılır.
    - Not:
    Eğer skorlar 1-5 arasıdaysa 1-3 negatif, 4-5 pozitif olarak işaretlenir ve bernoulli'ye uygun hale getirilebilir.
    Bu beraberinde bazı problemleri de getirir. Bu sebeple bayesian average rating yapmak gerekir.

    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    n = helpful_yes + helpful_no
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * helpful_yes / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)


In [42]:
# score_pos_neg_diff
df["score_pos_neg_diff"] = df.apply(lambda x: score_pos_neg_diff(x["helpful_yes"],
                                                                             x["helpful_no"]), axis=1)

In [50]:
df.sort_values("score_pos_neg_diff", ascending=False).head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,68,1884,0.96634,0.95754
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,126,1442,0.92562,0.91214
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.00000,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,77,1351,0.94884,0.93652
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.00000,"Warning, read this!",1346544000,2012-02-09,1033,422,495,73,349,0.85253,0.81858
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.00000,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,777,112,139,27,85,0.80576,0.73214


In [52]:
df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"], x["helpful_no"]), axis=1)

In [53]:
df.sort_values("score_average_rating", ascending=False).head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
4277,A35KXSU6AD1481,B007WTAJTO,S. Q.,"[1, 1]",I have a galaxy note II and after rooting I no...,5.00000,Perfect!!,1355875200,2012-12-19,719,1,1,0,1,1.00000,0.20655
2881,A3VSG5X7GPNNW6,B007WTAJTO,Lou Thomas,"[1, 1]",The Nexus One is listed as supporting a maximu...,5.00000,Nexus One Loves This Card!,1349049600,2012-01-10,1063,1,1,0,1,1.00000,0.20655
1073,A2ZXEKQ2OBZLEE,B007WTAJTO,C. Sanchez,"[1, 1]",I used it with my Samsung S4 and it works grea...,5.00000,Tons of space for phone,1376352000,2013-08-13,482,1,1,0,1,1.00000,0.20655
445,AIWBDRNBODLEA,B007WTAJTO,"Apache ""Elizabeth""","[1, 1]",This is exactly what I was looking for to upgr...,4.00000,Amazon Great Prices,1387324800,2013-12-18,355,1,1,0,1,1.00000,0.20655
3923,A2PH4RGYVR34L,B007WTAJTO,Rock Your Roots,"[1, 1]","It's a SanDisk, so what more is there to say? ...",5.00000,What more to say?,1388361600,2013-12-30,343,1,1,0,1,1.00000,0.20655


In [54]:
#### Step 3: Determine the top 20 reviews and interpret the results.

# - Sort and identify the top 20 reviews according to wilson_lower_bound.
# - Interpret the results.

df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)

In [55]:
df.sort_values("wilson_lower_bound", ascending=False).head(20)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,68,1884,0.96634,0.95754
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.00000,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,77,1351,0.94884,0.93652
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,126,1442,0.92562,0.91214
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.00000,"Warning, read this!",1346544000,2012-02-09,1033,422,495,73,349,0.85253,0.81858
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.00000,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,158,45,49,4,41,0.91837,0.80811
1835,A1J6VSUM80UAF8,B007WTAJTO,goconfigure,"[60, 68]",Bought from BestBuy online the day it was anno...,5.00000,I own it,1393545600,2014-02-28,283,60,68,8,52,0.88235,0.78465
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.00000,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,777,112,139,27,85,0.80576,0.73214
3807,AFGRMORWY2QNX,B007WTAJTO,R. Heisler,"[22, 25]",I bought this card to replace a lost 16 gig in...,3.00000,"Good buy for the money but wait, I had an issue!",1361923200,2013-02-27,649,22,25,3,19,0.88000,0.70044
4306,AOHXKM5URSKAB,B007WTAJTO,Stellar Eller,"[51, 65]","While I got this card as a ""deal of the day"" o...",5.00000,Awesome Card!,1339200000,2012-09-06,823,51,65,14,37,0.78462,0.67033
4596,A1WTQUOQ4WG9AI,B007WTAJTO,"Tom Henriksen ""Doggy Diner""","[82, 109]",Hi:I ordered two card and they arrived the nex...,1.00000,Designed incompatibility/Don't support SanDisk,1348272000,2012-09-22,807,82,109,27,55,0.75229,0.66359


In [ ]:
# Wisdom of Crowd: This concept refers to the tendency for the collective opinions or predictions
# of a group of individuals to be more accurate than individual opinions. When various individuals with
# different perspectives and knowledge come together, the collectively emerged view or prediction tends
# to be more accurate than that of any single individual. This phenomenon arises from the combination of
# diverse knowledge and experiences within the group and is often used for statistical predictions or decisions.

# Wilson Lower Bound: The Wilson Lower Bound is a statistical method used to calculate the confidence interval
# of the true value of an item with a certain proportion of positive feedback, using the binomial distribution.
# It is particularly used in cases where there is positive feedback, such as user reviews. This method can be used
# to determine the popularity or success of a product or service. The Wilson Lower Bound provides the lower limit
# of the true positive rate at a certain confidence level, allowing for a reliable estimation. It is commonly used,
# especially in situations with limited feedback.

# Social Proof: Social proof is a psychological phenomenon where people assume the actions of others in
# an attempt to reflect correct behavior for a given situation. In other words,
# it's the influence that the actions and attitudes of others have on our own behavior. When individuals
# are uncertain about how to behave in a particular situation, they often look to others for guidance.
# This can manifest in various forms, such as observing others' actions, following trends, or seeking validation from peers.
# Social proof plays a significant role in decision-making processes, consumer behavior, and social interactions.
# It's commonly leveraged in marketing and advertising to persuade individuals to adopt certain behaviors or
# purchase products/services by showcasing testimonials, user reviews, endorsements, or statistics indicating
# popularity or approval by others. Overall, social proof acts as a powerful motivator, shaping our perceptions
# and influencing our actions based on the behavior of those around us.